In [ ]:
!pip install openrouteservice geopandas mapclassify folium==0.14.0 pyogrio

In [1]:
import openrouteservice

client = openrouteservice.Client(key='') # Fill in api key here

In [2]:
import json
with open("mrt.geojson") as f:
    mrt_geojson = json.load(f)

with open("lrt1.geojson") as f:
    lrt1_geojson = json.load(f)
  
with open("lrt2.geojson") as f:
    lrt2_geojson = json.load(f)

In [23]:
import geopandas as gpd
import pandas as pd
import pyogrio

# https://openrouteservice.org/restrictions/
# https://openrouteservice.org/dev/#/api-docs/v2/isochrones/{profile}/post 
time = [3*60, 6*60, 9*60, 12*60, 15*60] # max of 10 intervals
profile = 'foot-walking'
max_query_size = 5 # max 5 for isochrones
def isochrones(geojson):
    locations = [ feature['geometry']['coordinates'] for feature in geojson['features']]
    gdfs = []
    for i in range(0, len(locations), max_query_size):
        chunk = locations[i:i+max_query_size]
        results = openrouteservice.isochrones.isochrones(client, locations=chunk, profile=profile,range=time)
        gdf = gpd.GeoDataFrame.from_features(results["features"])
        gdfs.append(gdf)
    return pd.concat(gdfs)

In [9]:
mrt_gdf = isochrones(mrt_geojson)
pyogrio.write_dataframe(mrt_gdf, "mrt_isochrones.geojson", driver="GeoJSON")

In [ ]:
lrt1_gdf = isochrones(lrt1_geojson)
pyogrio.write_dataframe(lrt1_gdf, "lrt1_isochrones.geojson", driver="GeoJSON")

In [22]:
lrt2_gdf = isochrones(lrt2_geojson)
pyogrio.write_dataframe(lrt2_gdf, "lrt2_isochrones.geojson", driver="GeoJSON")

In [24]:
all = pd.concat([mrt_gdf, lrt1_gdf, lrt2_gdf])

In [111]:
all

,geometry,group_index,value,center
0,"POLYGON ((121.03081 14.65373, 121.03135 14.652...",0,180.0,"[121.03226031541993, 14.652202085132709]"
1,"POLYGON ((121.02939 14.65535, 121.02958 14.654...",0,360.0,"[121.03226031541993, 14.652202085132709]"
2,"POLYGON ((121.02727 14.65584, 121.02906 14.652...",0,540.0,"[121.03226031541993, 14.652202085132709]"
3,"POLYGON ((121.02501 14.65633, 121.02581 14.655...",0,720.0,"[121.03226031541993, 14.652202085132709]"
4,"POLYGON ((121.02282 14.65692, 121.02358 14.655...",0,900.0,"[121.03226031541993, 14.652202085132709]"
...,...,...,...,...
10,"POLYGON ((120.98084 14.60378, 120.98176 14.602...",2,180.0,"[120.98336495641382, 14.60358554379944]"
11,"POLYGON ((120.97870 14.60462, 120.97908 14.603...",2,360.0,"[120.98336495641382, 14.60358554379944]"
12,"POLYGON ((120.97665 14.60525, 120.97731 14.603...",2,540.0,"[120.98336495641382, 14.60358554379944]"
13,"POLYGON ((120.97449 14.60607, 120.97515 14.604...",2,720.0,"[120.98336495641382, 14.60358554379944]"


In [112]:
all.explore()

In [34]:
# Merge geometry with same value/time interval
merged_rows = []
for time in all.value.unique():
    subset = all[all.value == time]
    merged_rows.append({'geometry':subset.geometry.unary_union, 'time': time})

merged_gdf = gpd.GeoDataFrame(merged_rows)

In [105]:
# Clean geometries by removing area that are contained in smaller intervals. This allows styling to be 
last_geom= None
geoms_clean = []
for idx, row in merged_gdf.iterrows():
    if idx != 0:
        diff = row.geometry.difference(last_geom)
        geoms_clean.append({'geometry': diff, 'time':row.time })
    else:
        geoms_clean.append({'geometry': row.geometry, 'time':row.time})
    last_geom = row.geometry
clean_gdf = gpd.GeoDataFrame(geoms_clean)

In [106]:
clean_gdf.explore()

In [108]:
clean_gdf

,geometry,time
0,"MULTIPOLYGON (((120.99929 14.55494, 120.99938 ...",180.0
1,"MULTIPOLYGON (((121.00036 14.55644, 121.00099 ...",360.0
2,"MULTIPOLYGON (((121.00246 14.55218, 121.00198 ...",540.0
3,"MULTIPOLYGON (((121.00532 14.55221, 121.00528 ...",720.0
4,"MULTIPOLYGON (((121.04435 14.55551, 121.04431 ...",900.0


In [110]:
pyogrio.write_dataframe(clean_gdf, "final.geojson", driver="GeoJSON")